In [22]:
import pandas as pd
#dfC = pd.read_csv('dc10.csv')
ds = pd.read_csv('df10.csv')

,Unnamed: 0,proc_combos,pct_composition
0,18,98941@99212,5.592778e-03
1,30,98940@99212,3.452621e-03
2,79,99212,9.184727e-04
3,81,97014@98941@99212,8.643128e-04
4,101,98941@98943@99212,6.681587e-04
...,...,...,...
8903,89170,99212@S5000@S9999,1.403105e-07
8904,89171,99212@S8990@S8990@S8990,1.403105e-07
8905,89172,99212@S8990@S8990@S8990@S8990@S8990@S8990,1.403105e-07
8906,89173,99212@S8990@S8990@S8990@S8990@S8990@S8990@S899...,1.403105e-07


,Unnamed: 0,hcpcs_code,mod_values,diag_values,place_of_service,pat_gender,age_bin,line_cnt,prov_cnt,specialty_id,pct_composition,pct_peers,Count,DrID,DrCount
0,435871,97032,None,M9901@M9902,11.0,F,Adult,1264,59,25,0.005677,0.000863,1,0,0.005677
1,442223,97032,None,M9903,11.0,F,Adult,1107,40,25,0.004972,0.000585,1,0,0.004972
2,442226,97032,None,M9903,11.0,M,Adult,1081,56,25,0.004856,0.000820,1,0,0.004856
3,434223,97032,None,M9901@M542,11.0,F,Adult,915,34,25,0.004110,0.000498,1,0,0.004110
4,432849,97032,None,M9901,11.0,F,Adult,915,56,25,0.004110,0.000820,1,0,0.004110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,430548,97032,None,M545@M9904@M542@M9903,11.0,F,Adult,25,3,25,0.000112,0.000044,1,199,0.000112
199996,437208,97032,None,M9901@M9902@S134XXA,11.0,F,Minor,25,2,25,0.000112,0.000029,1,199,0.000112
199997,427434,97032,None,M542@M62830,11.0,F,Adult,25,7,25,0.000112,0.000102,1,199,0.000112
199998,450926,97032,None,M9907,11.0,F,Adult,25,6,25,0.000112,0.000088,1,199,0.000112


In [25]:
def combo(cpt,dfa):
    df = dfa    
    dfProcCombos = pd.read_csv('proc_combos.csv')
    dfProcCombos['proc_values'] = dfProcCombos['proc_values'].astype(str)
    dfProcCombos['sorted'] = dfProcCombos['proc_values'].str.split('@')
    dfProcCombos['sorted'] = dfProcCombos['sorted'].apply(sorted)
    dfProcCombos['sorted'] = dfProcCombos['sorted'].transform(lambda x: '@'.join(x))
    dfProcCombos['proc_values'] = dfProcCombos['sorted']
    dfProcCombos = dfProcCombos.drop('sorted', axis=1)
    dfProcCombos = dfProcCombos[dfProcCombos['proc_values'].str.contains(cpt)]
    lngClaimCnt = len(dfProcCombos['claim_id'].unique())
    lngLineCnt = dfProcCombos.shape[0]
    dfComboCnts = dfProcCombos.groupby('proc_values', as_index=False).size()
    dfProcCombos = dfProcCombos.merge(dfComboCnts)

    dfProcCombos['line_pct_composition'] = dfProcCombos['size']/lngLineCnt
    dfClaimCnts = dfProcCombos[['proc_values', 'claim_id']].drop_duplicates().groupby('proc_values', as_index=False).size()
    dfProcCombos = dfProcCombos.merge(dfClaimCnts.rename(columns={'size': 'claim_size'}))
    dfProcCombos['claim_pct_composition'] = dfProcCombos['claim_size']/lngClaimCnt
    dfProcCombos['synthetic_lines'] = dfProcCombos['line_pct_composition']*df.shape[0]
    dfProcCombos['lines_to_claims_ratio'] = dfProcCombos['size']/dfProcCombos['claim_size']
    dfProcCombos['synthetic_claims'] = dfProcCombos['synthetic_lines']/dfProcCombos['lines_to_claims_ratio']
    dfProcCombos.drop_duplicates(subset='proc_values')
    return dfProcCombos

In [26]:
combo()
#generate claim ID first
#maybe add another column to below that would be proc values then randomly assign based on proc values based on hcpcs code
#use number of lines as percent and make sure that hcpcs code is in something like  dfProcCombos = dfProcCombos[dfProcCombos['proc_values'].str.contains(cpt)]\
# need to figure out a way around for loo

,claim_id,proc_values,size,line_pct_composition,claim_size,claim_pct_composition,synthetic_lines,lines_to_claims_ratio,synthetic_claims
0,116734371,97032@98940,61150,0.096265,30575,0.157277,19252.959965,2.0,9626.479983
1,116734371,97032@98940,61150,0.096265,30575,0.157277,19252.959965,2.0,9626.479983
2,117699228,97032@98940,61150,0.096265,30575,0.157277,19252.959965,2.0,9626.479983
3,117699228,97032@98940,61150,0.096265,30575,0.157277,19252.959965,2.0,9626.479983
4,117788409,97032@98940,61150,0.096265,30575,0.157277,19252.959965,2.0,9626.479983
...,...,...,...,...,...,...,...,...,...
635222,192989620,97012@97032@97032@97032@97032@97032@97530@9753...,20,0.000031,1,0.000005,6.296962,20.0,0.314848
635223,192989620,97012@97032@97032@97032@97032@97032@97530@9753...,20,0.000031,1,0.000005,6.296962,20.0,0.314848
635224,192989620,97012@97032@97032@97032@97032@97032@97530@9753...,20,0.000031,1,0.000005,6.296962,20.0,0.314848
635225,192989620,97012@97032@97032@97032@97032@97032@97530@9753...,20,0.000031,1,0.000005,6.296962,20.0,0.314848
